In [11]:
import requests
import json


In [12]:
resp = requests.get("https://api.elsevier.com/content/abstract/eid/2-s2.0-85114075647",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': '6545798661bccd5ab7ef446593a92c2f'})
#,'X-ELS-Insttoken':'c2eb041217badb7ae8addef94d71947e'

In [13]:
# resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][0]['ref-fulltext']

'Zhu, N.; Zhang, D.; Wang, W.; Li, X.; Yang, B.; Song, J.; Zhao, X.; Huang, B.; Shi, W.; Lu, R.; et al. A Novel Coronavirus from Patients with Pneumonia in China, 2019. N. Engl. J. Med. 2020, 382, 727–733. [CrossRef] [PubMed]'

### Required format

ref-authors(first author),ref-publicationyear, ref-title, ref-sourcetitle,ref-volisspag,727-733

Zhu N, 2020, A Novel Coronavirus from Patients with Pneumonia in China, 2019, N. Engl. J. Med, 382, 727-733

In [4]:
resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][0]

{'@aii:was-generated-by': 'http://data.elsevier.com/art/structure-references art:version v1.0.181',
 'ref-fulltext': 'Zhu, N.; Zhang, D.; Wang, W.; Li, X.; Yang, B.; Song, J.; Zhao, X.; Huang, B.; Shi, W.; Lu, R.; et al. A Novel Coronavirus from Patients with Pneumonia in China, 2019. N. Engl. J. Med. 2020, 382, 727–733. [CrossRef] [PubMed]',
 '@id': '1',
 'ref-info': {'ref-publicationyear': {'@first': '2020'},
  'ref-title': {'ref-titletext': 'A Novel Coronavirus from Patients with Pneumonia in China, 2019'},
  'refd-itemidlist': {'itemid': {'$': '85080034896', '@idtype': 'SGR'}},
  'ref-volisspag': {'voliss': {'@volume': '382'},
   'pagerange': {'@first': '727', '@last': '733'}},
  'ref-text': '[CrossRef] [PubMed]',
  'ref-authors': {'author': [{'@seq': '1',
     'ce:initials': 'N.',
     '@_fa': 'true',
     'ce:surname': 'Zhu',
     'ce:indexed-name': 'Zhu N.'},
    {'@seq': '2',
     'ce:initials': 'D.',
     '@_fa': 'true',
     'ce:surname': 'Zhang',
     'ce:indexed-name': 'Zha

In [5]:
print (json.dumps(resp.json(),
                 sort_keys=True,
                 indent=4, separators=(',', ': ')))#Should populate some results

{
    "abstracts-retrieval-response": {
        "affiliation": {
            "@href": "https://api.elsevier.com/content/affiliation/affiliation_id/60000964",
            "@id": "60000964",
            "affiliation-city": "Liege",
            "affiliation-country": "Belgium",
            "affilname": "Universit\u00e9 de Li\u00e8ge"
        },
        "authkeywords": {
            "author-keyword": [
                {
                    "$": "Antigen detection",
                    "@_fa": "true"
                },
                {
                    "$": "COVID-19",
                    "@_fa": "true"
                },
                {
                    "$": "Rapid diagnostic test",
                    "@_fa": "true"
                },
                {
                    "$": "SARS-CoV-2",
                    "@_fa": "true"
                }
            ]
        },
        "authors": {
            "author": [
                {
                    "@_fa": "true",
               

### Use the updated excel file here

In [6]:
import pandas as pd
import numpy as np
combined = pd.read_excel('scopus_master_with_references_formatted_2_230609_month_year.xlsx')

In [7]:
combined.eid.values

array(['2-s2.0-85099799917', '2-s2.0-85114075647', '2-s2.0-85089978670',
       ..., '2-s2.0-85125696300', '2-s2.0-85124829934',
       '2-s2.0-85128229965'], dtype=object)

### Change this to 'References_formatted' in all the parts after this. Make sure to have  ';' separator between each references

In [15]:
combined['References'] = np.nan

In [18]:
dummy=''
for i in combined[combined['References'].isna()]['eid'].values:
    print(i)
    resp = requests.get(f"https://api.elsevier.com/content/abstract/eid/{i}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': 'd886e786b7cff90747dfce85a88d35ea'
                             })
    #'X-ELS-Insttoken':'c2eb041217badb7ae8addef94d71947e'
    if 'abstracts-retrieval-response' not in resp.json().keys():
        continue
    temporary = resp.json()['abstracts-retrieval-response']['item']
    if temporary is not None:
        if temporary['bibrecord'] is not None:
            temporary1 = temporary['bibrecord']
            if temporary1['tail'] is not None:
                temporary2 = temporary1['tail']
                if temporary2['bibliography'] is not None:
                    temporary3 = temporary2['bibliography']

                    refcount = int(temporary3['@refcount'])
                    full_text = temporary3['reference']
                    if i =='2-s2.0-85106644551':
                        continue
                    temp=''
                    if refcount==1:
                        temp = full_text['ref-fulltext'].replace(';,',',').replace(',;',',').replace(';',',')
                    else:        
                        for j in range(refcount):
                            if temp!='':
                                temp+=';'
                            
                            print(j)
#                             print(full_text[j]['ref-fulltext'])
                            dummy = full_text[j]['ref-fulltext'].replace(';,',',').replace(',;',',').replace(';',',')
                            temp+=dummy
    else:
        continue
#     print('$$$$$$$$$$$$$',temp)
    combined.loc[combined.eid.values==i,'References'] = temp                

2-s2.0-85106644551
2-s2.0-85125550067
2-s2.0-85126347331
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
2-s2.0-85129841820
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
2-s2.0-85130749799
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2-s2.0-85128381684
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2-s2.0-85126387007
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2-s2.0-85125148030
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
2-s2.0-85129825425
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
2-s2.0-85130047239
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
2-s2.0-85126422014
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
2-s2.0-85127907873
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85129748790
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
2-s2.0-85128804417
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
2-s2.0-85128877068
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2-s2.0-85123960827
0
1
2
3

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2-s2.0-85134854560
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
2-s2.0-85122469958
0
1
2
3
4
2-s2.0-85121332405
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
2-s2.0-85117853200
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
2-s2.0-85096100209
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85094123297
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2-s2.0-85092461354
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
2-s2.0-85096429640
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
2-s2.0-85089480804
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2-s2.0-85089443141
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2-s2.0-85104260232
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
2-s2.0-85102940031
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85100488406
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
2-s2.0-85087922288
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2-s2.0-85086167894
0
1
2
3
4
5
6
7


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2-s2.0-85135031221
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
2-s2.0-85130438417
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
2-s2.0-85135070919
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
2-s2.0-85130402696
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2-s2.0-85133681473
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2-s2.0-85135065996
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85132203852
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2-s2.0-85131903369
0
1
2
3
4
5
6
7
8
9
2-s2.0-85145853735
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85135282048
0
1
2
3
4
5
6
7
8
9
10
11
12
2-s2.0-8513442687

...

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2-s2.0-85126409165
0
1
2
3
4
5
6
7
8
9
10
11
12
13
2-s2.0-85128665905
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
2-s2.0-85124002187
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2-s2.0-85123917755
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2-s2.0-85124956935
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
2-s2.0-85125836634
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
2-s2.0-85125363138
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
2-s2.0-85132080485
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
2-s2.0-85123996964
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
2-s2.0-85127488108
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
2-s2.0-85127478869
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
2-s2.0-85129882828
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
2-s2.0-85127413112
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
2-s2.0-85123433766
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
2-s2.0-85124613350
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
2-s2.0-8512

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
2-s2.0-85138630148
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2-s2.0-85138612084
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
2-s2.0-85138541838
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2-s2.0-85138533453
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
2-s2.0-85138385078
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
2-s2.0-85138413556
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2-s2.0-85138408295
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
2-s2.0-85126867003
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
2-s2.0-85128849582
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
2-s2.0-85124544320
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
2-s2.0-85127409013
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
2-s2.0-85123923512
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
2-s2.0-85130033769
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
2-s2.0-85083865989
0
1
2
3
4
5
6
7
8
9
10
2-s2.0-85082434854
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2-s2.0-85083277461
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2-s2.0-85083162042
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
2-s2.0-85136935422
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
2-s2.0-85136909800
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
2-s2.0-85136538838
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2-s2.0-85136339549
0
1
2
3
4
5
6
7
8
9
10
2-s2.0-85136209283
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
2-s2.0-85136154774
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2-s2.0-85138943169
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
2-s2.0-85109298975
0
1
2
3
4
5
6

In [19]:
combined[combined['References'].isna()]

,ID,STUDY TITLE/NAME,STUDY DESCRIPTION,STUDY METHODOLOGY,PUBLICATION DATE,SAMPLE SIZE,TEST MANUFACTURER,TEST MANUFACTURER.1,TEST MANUFACTURER.2,LINK TO PUBLICATION,...,openaccessFlag,Affliation-name,fund-sponsor,link ref=self,link ref=scopus,link ref=scopus-citedby,dc:title_lower,match_score,match,References
375,467,Convalescent plasma in patients admitted to ho...,We aimed to evaluate the safety and efficacy o...,Test Validation,2021-05-01 00:00:00,11558,Other - Serology,NaN,NaN,https://www.thelancet.com/article/S0140-6736(2...,...,True,NaN,Bill and Melinda Gates Foundation,https://api-elsevier-com.ezproxy1.lib.asu.edu/...,https://www-scopus-com.ezproxy1.lib.asu.edu/in...,https://www-scopus-com.ezproxy1.lib.asu.edu/in...,convalescent plasma in patients admitted to ho...,1.0,1,NaN
2795,2891,High SARS-CoV-2 Prevalence among Healthcare Wo...,The aim of the study was to estimate the SARS-...,Community/Epidemiological,February 2022,783,Euroimmun,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,...,True,"Aix Marseille Université,Viedma Hospital,María...",NaN,https://api-elsevier-com.ezproxy1.lib.asu.edu/...,https://www-scopus-com.ezproxy1.lib.asu.edu/in...,https://www-scopus-com.ezproxy1.lib.asu.edu/in...,high sars-cov-2 prevalence among healthcare wo...,1.0,1,NaN


In [8]:
scopus_api_results = pd.read_csv('scopus_Api_results_affli_5cols_virinchi_test_4cols.csv')

In [11]:
scopus_api_results

,eid,affiliation-city,affiliation-country,affiliation-afid,affiliation-url,affiliation-name-variant
0,2-s2.0-85148333013,"Hangzhou,Hangzhou,Hangzhou","China,China,China","6,011,842,860,003,970,000,000,000",https://api.elsevier.com/content/affiliation/a...,NaN
1,2-s2.0-85147796186,"Zhengzhou,Taiyuan","China,Hong Kong,China","600,185,546,001,434,000,000,000",https://api.elsevier.com/content/affiliation/a...,NaN
2,2-s2.0-85146193592,"Koya,Baghdad,Dublin","Iraq,Iraq,Ireland","601,041,836,007,115,000,000,000",https://api.elsevier.com/content/affiliation/a...,NaN
3,2-s2.0-85129779903,"Shenzhen,Shanghai,Lahore,Lahore,Abha","China,China,Pakistan,Pakistan,Saudi Arabia","6,027,196,160,123,530,000,000,000,000,000,000,...",https://api.elsevier.com/content/affiliation/a...,NaN
4,2-s2.0-85152053656,Jakarta,Indonesia,60069400,https://api.elsevier.com/content/affiliation/a...,NaN
...,...,...,...,...,...,...
147068,2-s2.0-85125688653,"Novara,Pavia,Vercelli,Novara","Italy,Italy,Italy,Italy","600,295,356,002,655,000,000,000,000,000,000",https://api.elsevier.com/content/affiliation/a...,NaN
147069,2-s2.0-85125688074,Istanbul,Turkey,60198715,https://api.elsevier.com/content/affiliation/a...,NaN
147070,2-s2.0-85125687847,"Recife,Recife","Brazil,Brazil","6,003,148,260,021,980",https://api.elsevier.com/content/affiliation/a...,NaN
147071,2-s2.0-85125682723,"Trabzon,Trabzon,Trabzon","Turkey,Turkey,Turkey","6,008,820,660,008,910,000,000,000",https://api.elsevier.com/content/affiliation/a...,NaN


In [10]:
scopus_api_results[["affiliation-country","affiliation-city","affiliation-afid","affiliation-name-variant"]].dropna()

,affiliation-country,affiliation-city,affiliation-afid,affiliation-name-variant


In [12]:
combined["eid"]

0       2-s2.0-85099799917
1       2-s2.0-85114075647
2       2-s2.0-85089978670
3       2-s2.0-85108303432
4       2-s2.0-85128295641
               ...        
3735    2-s2.0-85121957869
3736    2-s2.0-85121957763
3737    2-s2.0-85125696300
3738    2-s2.0-85124829934
3739    2-s2.0-85128229965
Name: eid, Length: 3740, dtype: object

In [13]:
combined_scopus_api = combined.merge(scopus_api_results, on = "eid", how = "left")

In [14]:
combined_scopus_api.to_excel('scopus_master_with_references_formatted_2_230614_month_year_aff_names.xlsx',index=False)

C:\Users\sjunuthu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://api.elsevier.com/content/affiliation/affiliation_id/60176939,https://api.elsevier.com/content/affiliation/affiliation_id/60170515,https://api.elsevier.com/content/affiliation/affiliation_id/60162089,https://api.elsevier.com/content/affiliation/affiliation_id/60162085,https://api.elsevier.com/content/affiliation/affiliation_id/60160274,https://api.elsevier.com/content/affiliation/affiliation_id/60138366,https://api.elsevier.com/content/affiliation/affiliation_id/60120017,https://api.elsevier.com/content/affiliation/affiliation_id/60117212,https://api.elsevier.com/content/affiliation/affiliation_id/60111558,https://api.elsevier.com/content/affiliation/affiliation_id/60111140,https://api.elsevier.com/content/affiliation/affiliation_id/60029329,https://api.elsevier.com/content/affiliation/affiliation_id/60027676,https://api.elsevier.com/content/affiliation/affiliation_id/60026851,

In [180]:
combined.to_excel('scopus_master_combined_with_match_1705.xlsx',index=False)